In [ ]:
from datasets import load_dataset
import pandas as pd
from huggingface_hub import login
import random
import re

login(token="<your_token>")  # 로그인 위젯 실행 (토큰 필요)


In [ ]:
dataset = load_dataset('<your_dataset>')

In [ ]:
dataset.set_format(type='pandas')

df = dataset['train'][:]

In [ ]:
substitutions = {
    'A': (r'A\.(.*?)B\.', 'A. [공백] B.'),
    'B': (r'B\.(.*?)C\.', 'B. [공백] C.'),
    'C': (r'C\.(.*?)D\.', 'C. [공백] D.'),
    'D': (r'D\.(.*)', 'D. [공백]')
}

options = ['A', 'B', 'C', 'D']

PoE_Q = []
PoE_A = []

In [ ]:
# 1차 필터링
for ans, ques in zip(df['answer'], df['question']):
    answer_last_char = ans[-1]
    if answer_last_char in options:
        # 다른 선택지 문자가 포함되지 않은지 검사
        if not any((ch + ".") in ans for ch in options if ch != answer_last_char):
            PoE_A.append(ans)
            PoE_Q.append(ques)

df_poe = pd.DataFrame({'question': PoE_Q, 'answer': PoE_A})

In [ ]:
poe_answer = []
poe_question = []

# 2차 처리: 랜덤으로 2개 선택 후, 해당 부분 공백 처리
for q, a in zip(df_poe['question'], df_poe['answer']):
    answer_last_char = a[-1]
    remaining_letters = [x for x in options if x != answer_last_char]
    random_choices = random.sample(remaining_letters, 2)
    random_choices.sort()

    modified_text = q
    # 선택한 두 글자에 대해 순차적으로 정규식 치환
    for ch in random_choices:
        pattern, repl = substitutions[ch]
        modified_text = re.sub(pattern, repl, modified_text)

    poe_answer.append(a)
    poe_question.append(modified_text)


In [ ]:
real_df_poe = pd.DataFrame({'question': poe_question, 'answer': poe_answer})
real_df_poe.to_parquet('train-00000-of-00001.parquet', index=False)